# Seperate

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
berk.csv
berk-test.csv
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
relation_array.npy
samples
state_transition_matrix.npy


In [2]:
labels_list_file = r"/home/anhaoran/data/pedestrian_attributes_PETA/PETA/labels.txt" 
labels_list_data = open(labels_list_file)
lines = labels_list_data.readlines()
attributes_list = []
for line in lines:
    data = line.split()
    #print(attr)
    attributes_list.append(data[1])
print("The total attributes labeled: ", len(attributes_list))
print(attributes_list)

The total attributes labeled:  65
['accessoryHeadphone', 'personalLess15', 'personalLess30', 'personalLess45', 'personalLess60', 'personalLarger60', 'carryingBabyBuggy', 'carryingBackpack', 'hairBald', 'footwearBoots', 'lowerBodyCapri', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'lowerBodyCasual', 'upperBodyCasual', 'personalFemale', 'carryingFolder', 'lowerBodyFormal', 'upperBodyFormal', 'accessoryHairBand', 'accessoryHat', 'lowerBodyHotPants', 'upperBodyJacket', 'lowerBodyJeans', 'accessoryKerchief', 'footwearLeatherShoes', 'upperBodyLogo', 'hairLong', 'lowerBodyLongSkirt', 'upperBodyLongSleeve', 'lowerBodyPlaid', 'lowerBodyThinStripes', 'carryingLuggageCase', 'personalMale', 'carryingMessengerBag', 'accessoryMuffler', 'accessoryNothing', 'carryingNothing', 'upperBodyNoSleeve', 'upperBodyPlaid', 'carryingPlasticBags', 'footwearSandals', 'footwearShoes', 'hairShort', 'lowerBodyShorts', 'upperBodyShortSleeve', 'lowerBodyShortSkirt', 'footwearSneakers', 'footwearStockin

In [7]:
low_level = [27, 32, 50, 56]#, 61, 62, 63, 64
mid_level = [0, 6, 7, 8, 9, 11, 12, 13, 17, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 33, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 60]
high_level = [1, 2, 3, 4, 5, 10, 14, 15, 16, 18, 19, 31, 34, 40]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)
data_dict = {"attributes": attributes_list_hiar}

The total attributes labeled:  61
['upperBodyLogo', 'lowerBodyThinStripes', 'upperBodyThinStripes', 'upperBodyThickStripes', 'accessoryHeadphone', 'carryingBabyBuggy', 'carryingBackpack', 'hairBald', 'footwearBoots', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'carryingFolder', 'accessoryHairBand', 'accessoryHat', 'lowerBodyHotPants', 'upperBodyJacket', 'lowerBodyJeans', 'accessoryKerchief', 'footwearLeatherShoes', 'hairLong', 'lowerBodyLongSkirt', 'upperBodyLongSleeve', 'carryingLuggageCase', 'carryingMessengerBag', 'accessoryMuffler', 'accessoryNothing', 'carryingNothing', 'upperBodyNoSleeve', 'carryingPlasticBags', 'footwearSandals', 'footwearShoes', 'hairShort', 'lowerBodyShorts', 'upperBodyShortSleeve', 'lowerBodyShortSkirt', 'footwearSneakers', 'footwearStocking', 'upperBodySuit', 'carryingSuitcase', 'lowerBodySuits', 'accessorySunglasses', 'upperBodySweater', 'lowerBodyTrousers', 'upperBodyTshirt', 'upperBodyOther', 'upperBodyVNeck', 'personalLess15', 'personalLe

# 9500+1900+7600

In [8]:
from keras.preprocessing import image

filename = r"../results/PETA.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_x = np.zeros((length, 160, 75, 3))
data_y = np.zeros((length, 61))
for i in range(length):
    #img = image.load_img(path + m)
    img = image.load_img(data[i, 0], target_size=(160, 75, 3))
    data_x[i] = image.img_to_array(img)
    data_y[i] = np.array(data[i, 1:1+61], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
X_test = data_x[11400:]
y_test = data_y[11400:]
y_test_hiar = data_y_hiar[11400:]
print("The shape of the X_test is: ", X_test.shape)
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the X_test is:  (7600, 160, 75, 3)
The shape of the y_test is:  (7600, 61)
The shape of the y_test is:  (7600, 61)


In [30]:
import tqdm
predictions_prob = np.load("../results/predictions/app.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar

"""
all_images = []
pre_flag = []
for i in tqdm.tqdm(range(11400, length)):
    all_images.append(data[i, 0])
    pre_flag.append(sum(1-predictions[i-11400]==label[i-11400]))
for i in range(7600):
    if pre_flag[i]:
        print(i)
attributes_list_hiar_plus = []
attributes_list_hiar_plus.append("image")
for i in tqdm.tqdm(attributes_list_hiar):
    attributes_list_hiar_plus.append(i)
title = pd.DataFrame(attributes_list_hiar_plus)
#title.to_csv("../results/title.csv")
#print(title)
all_data = np.hstack((np.asarray(all_images).reshape((-1, 1)), np.asarray(label, dtype="int64")))
#print(all_data)
attributes_pd = pd.DataFrame(all_data)
#attributes_pd.to_csv("../results/test_predictions.csv")
"""
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_gap&dense_binary61_gap&dense_final500iter_model_predictions_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.9293032168144003
0.9327963164331904
0.9565202854528555
0.9520830675172837
0.9542965185503481


# GoogLeNet

In [40]:
def mA_acc(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true[:])
    N = M - P
    TP = sum(y_pred[:]*y_true[:])
    TN = list(y_pred[:]+y_true[:] == 0).count(True)
    if P != 0:
        res += TP/P + TN/N
    else:
        res += TN/N
    return res / 2

predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_acc(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# GoogLeNetSPP

In [41]:
predictions_prob = np.load("../results/predictions/GoogLeNetSPP_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_acc(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNetSPP": metric})

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# hiarGoogLeNet

In [42]:
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61v2_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_acc(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# hiarGoogLeNetSPP

In [43]:
predictions_prob = np.load("../results/predictions/hiarGoogLeNetSPP_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_acc(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNetSPP": metric})

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# hiarBayesGoogLeNet

In [44]:
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_predictions50_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_acc(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# hiarGoogLeNetWAM

In [45]:
predictions_prob = np.load("../results/predictions/hiarGoogLeNetWAM_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_acc(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNetWAM": metric})

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# DF

In [46]:
df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

,GoogLeNet,GoogLeNetSPP,hiarGoogLeNet,hiarGoogLeNetSPP,hiarBayesGoogLeNet,hiarGoogLeNetWAM
attributes,,,,,,
upperBodyLogo,0.571764,0.513158,0.834788,0.500000,0.872315,0.737312
lowerBodyThinStripes,0.500000,0.500000,0.499803,0.500000,0.500000,0.500000
upperBodyThinStripes,0.717954,0.679688,0.499801,0.500000,0.920880,0.904326
upperBodyThickStripes,0.740680,0.580513,0.492667,0.500000,0.932445,0.661158
accessoryHeadphone,0.700000,0.698146,0.496490,0.817815,0.839868,0.620000
carryingBabyBuggy,0.850015,0.680221,0.497430,0.945695,0.954124,0.823437
carryingBackpack,0.795231,0.748684,0.498081,0.856658,0.908456,0.871057
hairBald,0.796725,0.621411,0.497304,0.835130,0.852782,0.660221
footwearBoots,0.838168,0.712975,0.510645,0.856730,0.847759,0.803867


In [47]:
df.to_csv("../results/metrics/binary61_50_imagenet_mAacc7600.csv")

# Functions Def

In [5]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0).count(True)
        #print(P,',', N,',', TP,',', TN)
        if P != 0:
            res += TP/P + TN/N
        else:
            res += TN/N
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            M_ += 1
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def F1(prec_value, rec_value):
    return 2*prec_value*rec_value/(prec_value+rec_value)

In [23]:
selected = []
selected_ = [27,31,32,40,50,56,60,7,11,21,23,24,26,28,29,35,36,37,38,41,42,43,45,46,47,48,54,57,58,59,2,3,4,5,14,15,18,19,34]
###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###GoogLeNetSPP
predictions_prob = np.load("../results/predictions/GoogLeNetSPP_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]
print("GoogLeNetSPP")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNetSPP
predictions_prob = np.load("../results/predictions/hiarGoogLeNetSPP_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarGoogLeNetSPP")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_rl_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNetWAM
predictions_prob = np.load("../results/predictions/hiarGoogLeNetWAM_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarGoogLeNetWAM")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

GoogLeNet
0.7888605961602565
0.7663167003326307
0.8649646059350132
0.8296080951212723
0.8469175015386179
GoogLeNetSPP
0.7070567846166158
0.7002412968811633
0.8519803548060131
0.7614735688288526
0.8041884702658857
hiarGoogLeNet
0.4902996450309907
0.09198792006912718
0.16491731660152226
0.16245956747929888
0.16367921638789582
hiarGoogLeNetSPP
0.4874860968318052
0.09622666548060431
0.1736719179942816
0.16758798730508703
0.17057572094526177
hiarBayesGoogLeNet
0.5151173256858113
0.14212651660683107
0.24376514311711192
0.24237862722072734
0.243069907948961
hiarGoogLeNetWAM
0.4954829760206026
0.0947744492691699
0.1741642677497889
0.16253097998492236
0.1681466507542352


In [ ]:
low = [2, 7, 23, 27, 32, 34, 38, 40, 45, 46, 48, 56, 58]
mid = [ 3, 4, 5, 11, 14, 15, 21, 24, 26, 28, 31, 35, 36, 37, 41, 42, 43, 54, 57, 59, 60]
high = [18, 19, 29, 47, 50]
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61_cluster_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, [1,2,4,5,6,7,8,9,10,11,12,14,15,17,18,19,22,24,25,28,29,31,32,34,36,37,38,39,40,41,43,44,45,46,48,49,56,58,60]]
label = y_test[:, list(np.hstack((low, mid, high)))]
print("New_hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

# Bayes

In [49]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_cond_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions[:, selected]
label = label[:, selected]
print("hiarBayesGoogLeNet_cond")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_add_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions[:, selected]
label = label[:, selected]
print("hiarBayesGoogLeNet_add")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_pcm_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions[:, selected]
label = label[:, selected]
print("hiarBayesGoogLeNet_pcm")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_multi_loss_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions#[:, selected]
label = label#[:, selected]
print("hiarBayesGoogLeNet_multiloss")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_multi_mar_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions#[:, selected]
label = label#[:, selected]
print("hiarBayesGoogLeNet_multimar")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

***
0.9905263157894737
1.0
0.9975
0.9996052631578948
0.9976315789473684
0.9794736842105263
0.9481578947368421
0.9856578947368421
0.9880263157894736
0.9419736842105263
0.9996052631578948
0.9992105263157894
0.9917105263157895
0.9913157894736843
0.9730263157894737
0.9976315789473684
0.9843421052631579
0.9546052631578947
0.9976315789473684
0.9482894736842106
0.925921052631579
0.993421052631579
0.9722368421052632
0.9964473684210526
0.939078947368421
0.9888157894736842
0.9492105263157895
0.9201315789473684
0.9943421052631579
0.9852631578947368
0.9886842105263158
0.9288157894736843
0.9451315789473684
0.9897368421052631
0.9776315789473684
0.9911842105263158
0.9502631578947368
0.9906578947368421
0.9809210526315789
0.9967105263157895
0.9632894736842105
0.9788157894736842
0.9931578947368421
0.9347368421052632
0.9763157894736842
0.9297368421052632
0.9947368421052631
0.9888157894736842
0.8302631578947368
0.7488157894736842
0.8919736842105264
0.9332894736842106
0.9888157894736842
0.9605263157894737


In [14]:
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_multi_500_predictions_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions#[:, selected]
label = label#[:, selected]
print("hiarBayesGoogLeNet_multiloss")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_multi_mar_500_predictions_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("***")
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    print(accuracy_score(predictions[:, i], label[:, i]))
predictions = predictions#[:, selected]
label = label#[:, selected]
print("hiarBayesGoogLeNet_multimar")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

***
0.9938157894736842
1.0
0.9985526315789474
0.9997368421052631
0.9989473684210526
0.9973684210526316
0.9781578947368421
0.998421052631579
0.9942105263157894
0.9732894736842105
1.0
0.9997368421052631
0.9956578947368421
0.993421052631579
0.9877631578947368
0.9980263157894737
0.9898684210526316
0.9698684210526316
0.9992105263157894
0.970657894736842
0.9636842105263158
0.9975
0.9822368421052632
0.9969736842105263
0.9703947368421053
0.993421052631579
0.9692105263157895
0.9682894736842105
0.9968421052631579
0.9932894736842105
0.9935526315789474
0.96
0.9619736842105263
0.9939473684210526
0.9821052631578947
0.991578947368421
0.975
0.9930263157894736
0.9953947368421052
0.9988157894736842
0.9960526315789474
0.9875
0.9964473684210526
0.9586842105263158
0.9855263157894737
0.9660526315789474
0.9988157894736842
0.9993421052631579
0.969078947368421
0.9659210526315789
0.9905263157894737
0.9967105263157895
0.9976315789473684
0.9878947368421053
0.9907894736842106
0.96
0.9906578947368421
0.990526315789

# Hiar

In [32]:
for i in range(11):
    print(attributes_list[selected_[-i-1]])
###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected_]
predictions = predictions[:, -11:]
label = y_test[:, selected_]
label = label[:, -11:]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
predictions = predictions[:, -12:]
label = y_test_hiar[:, selected]
label = label[:, -12:]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###GoogLeNet_high
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_high_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar[:, -14:]

#print(predictions.shape)
#print(label.shape)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
    #print(metric[i])
metric = np.asarray(metric)
print("GoogLeNet_high")
predictions = predictions[:, [1,2,3,4,6,7,8,9,10,11,12,13]]
label = y_test_hiar[:, selected]
label = label[:, -12:]
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

upperBodyPlaid
personalMale
lowerBodyPlaid
upperBodyFormal
lowerBodyFormal
upperBodyCasual
lowerBodyCasual
personalLarger60
personalLess60
personalLess45
personalLess30
GoogLeNet
0.8112903372173093
0.8470203634085239
0.8926898276089017
0.9032381203106564
0.8979329965257871
hiarGoogLeNet
0.9070417566513179
0.9376256265664249
0.9592192982456146
0.9569150103110886
0.9580657687431984
hiarGoogLeNet_high
0.8403598898787799
0.8784912280701875
0.9224364035087735
0.914446492036331
0.9184240708658664


In [45]:
for i in range(4):
    print(attributes_list[selected_[i]])
    print(attributes_list_hiar[selected[i]])
###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected_]
predictions = predictions[:, :4]
label = y_test[:, selected_]
label = label[:, :4]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
predictions = predictions[:, :4]
label = y_test_hiar[:, selected]
label = label[:, :4]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###GoogLeNet_high
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_low_binary61_duan_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar[:, :4]

#print(predictions.shape)
#print(label.shape)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
    #print(metric[i])
metric = np.asarray(metric)
print("GoogLeNet_low")
predictions = predictions
label = y_test_hiar[:, selected]
label = label[:, :4]
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

upperBodyLogo
upperBodyLogo
lowerBodyThinStripes
lowerBodyThinStripes
upperBodyThinStripes
upperBodyThinStripes
upperBodyThickStripes
upperBodyThickStripes
GoogLeNet
0.6325996044922395
0.23049645390070922
0.6190476190476191
0.2631578947368421
0.3693181818181818
hiarGoogLeNet
0.8138395477300011
0.6866197183098591
0.8297872340425532
0.7894736842105263
0.8091286307053942
152.0
0.0
64.0
31.0
GoogLeNet_low
0.6094545966115643
0.028391167192429023
0.03011472275334608
0.2550607287449393
0.05386917486105173


In [42]:
for i in range(4, 28):
    print(attributes_list[selected_[i]])
    print(attributes_list_hiar[selected[i]])
###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected_]
predictions = predictions[:, 4:28]
label = y_test[:, selected_]
label = label[:, 4:28]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
predictions = predictions[:, 4:28]
label = y_test_hiar[:, selected]
label = label[:, 4:28]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

###GoogLeNet_high
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_mid_binary61_duan_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar[:, 4:28]

#print(predictions.shape)
#print(label.shape)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
    #print(metric[i])
metric = np.asarray(metric)
print("GoogLeNet_mid")
predictions = predictions[:, [2,5,10,12,13,15,16,17,20,21,22,23,25,26,27,29,30,31,32,37,39,40,41,42]]
label = y_test_hiar[:, selected]
label = label[:, 4:28]
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

carryingBackpack
carryingBackpack
carryingOther
carryingOther
accessoryHat
accessoryHat
upperBodyJacket
upperBodyJacket
lowerBodyJeans
lowerBodyJeans
footwearLeatherShoes
footwearLeatherShoes
hairLong
hairLong
lowerBodyLongSkirt
lowerBodyLongSkirt
carryingMessengerBag
carryingMessengerBag
accessoryMuffler
accessoryMuffler
accessoryNothing
accessoryNothing
carryingNothing
carryingNothing
carryingPlasticBags
carryingPlasticBags
footwearSandals
footwearSandals
footwearShoes
footwearShoes
lowerBodyShorts
lowerBodyShorts
upperBodyShortSleeve
upperBodyShortSleeve
lowerBodyShortSkirt
lowerBodyShortSkirt
footwearSneakers
footwearSneakers
accessorySunglasses
accessorySunglasses
lowerBodyTrousers
lowerBodyTrousers
upperBodyTshirt
upperBodyTshirt
upperBodyOther
upperBodyOther
upperBodyVNeck
upperBodyVNeck
GoogLeNet
0.8173555192517554
0.7131827627781632
0.8243103729187501
0.7808955498306636
0.8020158569923166
hiarGoogLeNet
0.9305702986406458
0.8921476513252881
0.9327203714160277
0.9198573058287441

In [12]:
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary61v2_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
label = y_test_hiar[:, selected]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarGoogLeNet


ZeroDivisionError: division by zero

In [14]:
def mA_acc(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true[:])
    N = M - P
    TP = sum(y_pred[:]*y_true[:])
    TN = list(y_pred[:]+y_true[:] == 0).count(True)
    if P != 0:
        res += TP/P + TN/N
    else:
        res += TN/N
    return res / 2

In [18]:
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]
print("GoogLeNet")
for i in range(predictions.shape[1]):
    print(mA_acc(predictions[:, i], label[:, i]))

GoogLeNet
0.7
0.8743415130523267
0.8805281432748537
0.8611788891034649
0.7925998887411198
0.8772022255925508
0.8500145687645687
0.7952307012274598
0.7967250727749194
0.8381675805791109
0.6233963445657704
0.824730923378401
0.4980911005792522
0.7741935483870968
0.7993133691073798
0.8076410586514373
0.8563516450740153
0.6045770709035777
0.8031343180084599
0.816447087451351
0.6992455418381345
0.795845233206995
0.882663232019883
0.8192129256813586
0.9144862178535484
0.8658536585365854
0.8346151832333404
0.5717642320085929
0.8193831096033302
0.862415031026783
0.8906922274577793
0.625
0.5
0.8309295516586093
0.8561307063142718
0.8499539039834938
0.851994593055118
0.8025187531607378
0.8160347290158746
0.6643149169052734
0.8050180231455132
0.7350749413488813
0.6378538409598504
0.8149903721585047
0.8187713189718002
0.7633414871027971
0.8859028960522013
0.854267195344109
0.8421637248934875
0.8403207405069112
0.7179538216560509
0.824617022915726
0.8012443912246265
0.7891434213194091
0.7713108023659

In [19]:
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary61_multi_500_predictions_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test_hiar#[:, selected_]
print("GoogLeNet")
for i in range(predictions.shape[1]):
    print(mA_acc(predictions[:, i], label[:, i]))

GoogLeNet
0.8615064446831364
0.5
0.4963815789473684
0.49794156706507303
0.49721337579617836
0.48533491874752277
0.4359674083749281
0.4886754966887417
0.5193556443556443
0.4045205199986399
0.49973042188974254
0.4980013783597519
0.48759145650953706
0.4806477093206951
0.44332144272691243
0.4970698577292793
0.4902761184123191
0.5006782482775394
0.5154127616976475
0.4753639430217784
0.5666919809841144
0.5155237186849123
0.5440821223211263
0.49983671121769563
0.5976011594141598
0.5394200271862256
0.4622100760750475
0.3423687225347537
0.4928631995477671
0.4899982042387887
0.5016527546977496
0.5299472539824991
0.5096464579756087
0.5802247268823906
0.48359268064260497
0.4834776438625677
0.6225442108073796
0.48729596721728063
0.47487035821757023
0.4899092107932643
0.47202655107017066
0.4893742895416302
0.4933485622144585
0.2494752472435176
0.5725867982386025
0.25039795534990406
0.5002126832788615
0.4957821286799474
0.4928165001537059
0.2911392900473563
0.41278326023391815
0.4427657560971566
0.49